In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 250
LEARNING_RATE = 5 * 1e-4
CUDA = torch.cuda.is_available()
PATH = 'AG-os-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc_v2 import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)

if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [ ]:
from ag_ep import *
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\250 (325s),  symKL_DB_eta: 426.972,  symKL_DB_z: 596.779,  loss: -1718.491,  ess: 2.703,  kl_eta_ex: 806.139,  kl_eta_in: 136.026,  kl_z_ex: 187.317,  kl_z_in: 32.957
epoch: 1\250 (325s),  symKL_DB_eta: 296.650,  symKL_DB_z: 446.616,  loss: -1303.040,  ess: 3.047,  kl_eta_ex: 517.116,  kl_eta_in: 124.662,  kl_z_ex: 129.397,  kl_z_in: 27.054
epoch: 2\250 (320s),  symKL_DB_eta: 229.583,  symKL_DB_z: 335.020,  loss: -1081.865,  ess: 3.352,  kl_eta_ex: 362.185,  kl_eta_in: 124.642,  kl_z_ex: 103.870,  kl_z_in: 23.354
epoch: 3\250 (311s),  symKL_DB_eta: 195.605,  symKL_DB_z: 263.110,  loss: -962.391,  ess: 3.604,  kl_eta_ex: 277.851,  kl_eta_in: 133.357,  kl_z_ex: 87.366,  kl_z_in: 19.906
epoch: 4\250 (320s),  symKL_DB_eta: 174.211,  symKL_DB_z: 198.464,  loss: -886.403,  ess: 3.891,  kl_eta_ex: 233.064,  kl_eta_in: 151.291,  kl_z_ex: 73.748,  kl_z_in: 16.716
epoch: 5\250 (323s),  symKL_DB_eta: 157.606,  symKL_DB_z: 151.187,  loss: -832.709,  ess: 4.153,  kl_eta_ex: 208.433,  kl_et

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)